# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][HW04] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from collections import Counter
from catboost import CatBoostClassifier

In [2]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
class DecisionTreeLeaf:
    def __init__(self, y):
        counter = Counter(y)
        most_common, _ = counter.most_common(1)[0]
        self.y = most_common
        total = len(y)
        self.probability = {label: (p / total) for label, p in counter.items()}
        
    def predict_proba(self,X):
        return self.probability
    
    def predict(self,X):
        return self.y
    
class Node:
    def __init__(self, split_dim, left = None, right = None):
        self.split_dim = split_dim
        self.left = left
        self.right = right
        
    def predict_proba(self, X):
        value = X[self.split_dim]
        if value == 0:
            return self.left.predict_proba(X)
        return self.right.predict_proba(X)
    
    def predict(self, X):
        if X[self.split_dim] == 0:
            return self.left.predict(X)        
        return self.right.predict(X)
    
        
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.criterion = criterion 
        self.n_X, self.n_features = X.shape
        if max_depth:
            self.max_depth = max_depth
        else:
            self.max_depth = float("inf")
            
        self.min_samples_leaf = min_samples_leaf
        
        if max_features == "auto":
            self.max_features = int(np.sqrt(self.n_features))
        else:
            self.max_features = max_features
        
        bagging_ind = np.random.choice(self.n_X, size  = self.n_X, replace=True)
        self.X = X[bagging_ind]
        self.y = y[bagging_ind]
        
        out_of_bag_ind = [x for x in range(self.n_X) if x not in bagging_ind]
        self.X_out  = X[out_of_bag_ind]
        self.y_out = y[out_of_bag_ind]
        
        self.root = self.build(np.arange(self.n_X))
        
    def build(self, indexes, depth = 0):
        if depth >= self.max_depth or len(indexes) <= self.min_samples_leaf:
            
            return DecisionTreeLeaf(self.y[indexes])

        split_gain, split_dim, indexes_left, indexes_right = self.best_split(indexes)


        if split_gain == 0:
            return DecisionTreeLeaf(self.y[indexes])

        left = self.build(indexes_left, depth + 1)
        right = self.build(indexes_right, depth + 1)

        return Node(split_dim, left, right)

    
    def best_split(self, indexes):
        best_gain = 0
        best_dim = 0
        min_leaf = self.min_samples_leaf
        best_indexes_left = []
        best_indexes_right = []
        
        if self.criterion == "gini":
            criterion = gini
        else:
            criterion = entropy
        
        
        self.random_features = np.random.choice(self.n_features, size=self.max_features, replace=False)
        
        for dim in self.random_features:
            X = self.X[indexes,dim]
                       
            indexes_left = indexes[X == 0]
            indexes_right = indexes[X == 1]                     
                
            if len(indexes_left) >= min_leaf and len(indexes_right) >= min_leaf:
            
                y = self.y
                split_gain = gain(y[indexes_left], y[indexes_right], criterion)
                if split_gain > best_gain:
                    best_gain = split_gain
                    best_dim = dim
                    best_indexes_left = indexes_left
                    best_indexes_right = indexes_right

        return best_gain, best_dim, best_indexes_left, best_indexes_right
        
    def predict_proba(self, X):
        proba = []
        for x in X:
            p = self.root.predict_proba(x)
            proba.append(p)
        return proba            
        
    def predict(self, X):
        pred = []
        for x in X:
            p = self.root.predict(x)
            pred.append(p)    
        return pred
    
    def feature_importance(self):
        importance = np.zeros(self.n_features)
        err_oob = 1 - np.mean(self.predict(self.X_out) == self.y_out)
        
        for i in range(self.n_features):
            X_shuffle = self.X_out.copy()
            np.random.shuffle(X_shuffle[:, i])
            err_oob_j = 1 - np.mean(self.predict(X_shuffle) == self.y_out)
            importance[i] = err_oob_j - err_oob

        return importance
    

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [4]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.estimators = []

    
    def fit(self, X, y):
        i = 0
        self.y_type = y.dtype
        self.n_features = X.shape[1]
        self.n_X = X.shape[0]
        while i < self.n_estimators:
            e = DecisionTree(X, y, self.criterion, self.max_depth, self.min_samples_leaf, self.max_features)
            i+=1
            self.estimators.append(e)
    
    def predict(self, X):
        pred = []
        res = np.empty((X.shape[0], self.n_estimators), dtype = self.y_type)
        i = 0
        for e in self.estimators: 
            res[:,i] = e.predict(X)
            i+=1
            
        for r in res:            
            p, _ = Counter(r).most_common(1)[0]
            pred.append(p)
        return pred
                

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [5]:
def feature_importance(rfc):
    importance = np.zeros((rfc.n_features, rfc.n_estimators))
    i = 0
    for e in rfc.estimators:
        importance[:,i] = e.feature_importance()
        i+=1
    return importance.mean(axis = 1)
    
    
def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1]
    return np.array(names)[idicies[:k]], importance[idicies[:k]], idicies

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [6]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [ 1.00647889e-04 -1.96372863e-03  1.66492368e-01  1.60051188e-01
  3.17101966e-01 -1.01424102e-03]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [7]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [8]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_t, y_age_train, y_age_t, y_sex_train, y_sex_t = train_test_split(X, y_age, y_sex, train_size=0.8)
X_val, X_test, y_age_val, y_age_test, y_sex_val, y_sex_test = train_test_split(X_t, y_age_t, y_sex_t, train_size=0.5)

#### Возраст

In [9]:
np.log2(X.shape[0])

12.952013164889486

In [10]:
# res = np.zeros((100000, 6))
# i = 0
# #1 - gini, 2 - entropy
# for crit in [1, 2]:
#     for d in [4, 5, 6]:
#         for msl in [5, 10, 30]:
#             for mf in [3, 6, 12]:
#                 for n_est in [10]:
#                     rfc = RandomForestClassifier(criterion=crit, max_depth=d, min_samples_leaf=msl,
#                                                        max_features=mf, n_estimators=n_est)
#                     rfc.fit(X_train, y_age_train)
#                     acc =  np.mean(rfc.predict(X_val) == y_age_val)
#                     res[i,:] =[crit, d , msl, mf, n_est , acc]
#                     i+=1
#                     print(f'criterion = {crit}, ',f'depth = {d},', f'min_samples_leaf = {msl},', f'max_features = {mf}, ',
#                          f'n_estimators = {n_est}')
#                     print("Accuracy:", np.mean(rfc.predict(X_val) == y_age_val))

# print(res[np.argmax(res[:,-1])])

In [11]:
# sort_res = res[np.argsort(res[:,-1])][::-1]
# sort_res[:10,:]

In [12]:
rfc = rfc = RandomForestClassifier(criterion='gini', max_depth=6, min_samples_leaf=10,
                                                       max_features="auto", n_estimators=30)
rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
importance = feature_importance(rfc)
k = 20
names, imp, ind = most_important_features(importance, features, k)
for i in range(k):
    print(str(i+1) + ".", names[i], imp[i])


Accuracy: 0.5863808322824716
Most important features:
1. styd.pozor 0.022962074479964404
2. 4ch 0.019798865311447356
3. dayvinchik 0.015666234458350744
4. ovsyanochan 0.015522930863289663
5. mudakoff 0.01482851597965982
6. rhymes 0.012802302419335125
7. pixel_stickers 0.006906646416633159
8. rapnewrap 0.006548263611826529
9. pozor 0.005930884118244524
10. tumblr_vacuum 0.005002558216380825
11. webestano 0.004991741867459121
12. memeboizz 0.004163205039195975
13. reflexia_our_feelings 0.004055947744669686
14. pravdashowtop 0.0037161070851349566
15. iwantyou 0.0030980743266318576
16. i_d_t 0.0027934147997337364
17. bog_memes 0.002772231921835781
18. ne1party 0.0022647906787495544
19. leprum 0.0022185511370564717
20. ne.poverish 0.0021339794735026675


In [13]:
k = X.shape[1]
names, imp, ind = most_important_features(importance, features, k)
for i in range(k):
    print(str(i+1) + ".", names[i], imp[i])

1. styd.pozor 0.022962074479964404
2. 4ch 0.019798865311447356
3. dayvinchik 0.015666234458350744
4. ovsyanochan 0.015522930863289663
5. mudakoff 0.01482851597965982
6. rhymes 0.012802302419335125
7. pixel_stickers 0.006906646416633159
8. rapnewrap 0.006548263611826529
9. pozor 0.005930884118244524
10. tumblr_vacuum 0.005002558216380825
11. webestano 0.004991741867459121
12. memeboizz 0.004163205039195975
13. reflexia_our_feelings 0.004055947744669686
14. pravdashowtop 0.0037161070851349566
15. iwantyou 0.0030980743266318576
16. i_d_t 0.0027934147997337364
17. bog_memes 0.002772231921835781
18. ne1party 0.0022647906787495544
19. leprum 0.0022185511370564717
20. ne.poverish 0.0021339794735026675
21. cook_bon 0.0021071137652326375
22. tysnm 0.0020145753905451294
23. pustota_diary 0.0018414337972708194
24. webmland 0.0017389322016542826
25. in.humour 0.0015667976021669177
26. borsch 0.0015049762336636426
27. rem_shkola 0.0014830383488445306
28. bratishkinoff 0.0013244579432361977
29. xfil

Проведем эксперимент! Давайте возьмем 80 фичей с наибольшей значимостью, а все остальные удалим. После 80 значимость почти нулевая у фичей, а у первых 80 значимость сильно не отличается, так что я не могу выделить какие-то определенные фичи, у которых значимость была бы сильно больше других. Так как другие фичи менее значимы, то предположим, что accuracy станет больше

In [14]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train1, X_test1, y_age_train1, y_age_test1, y_sex_train1, y_sex_test1 = train_test_split(X[:,ind[:80]], y_age, y_sex, train_size=0.8)

In [15]:
rfc = RandomForestClassifier(criterion='gini', max_depth=6, min_samples_leaf=30,
                                                       max_features = "auto", n_estimators=50)
rfc.fit(X_train1, y_age_train1)
print("Accuracy:", np.mean(rfc.predict(X_test1) == y_age_test1))

Accuracy: 0.6176656151419558


In [16]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_t, y_age_train, y_age_t, y_sex_train, y_sex_t = train_test_split(X, y_age, y_sex, train_size=0.8)

In [17]:
rfc = RandomForestClassifier(criterion='gini', max_depth=6, min_samples_leaf=30,
                                                       max_features = "auto", n_estimators=50)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_t) == y_sex_t))

Accuracy: 0.7829652996845425


Сильно лучше не стало. Возможно это вообще случайность. Так что не понятно, имело ли это большой смысл. Если взять первые 10, то лучше точно не становится. Так что есть смысл убирать только те фичи, где значимость равна 0. У всех остальных все-таки значимость примерно одинаковая и нельзя выделить какие-то фичи, у которых значимость сильно больше.

#### Пол

In [18]:
# res = np.zeros((100000, 6))
# i = 0
# #1 - gini, 2 - entropy
# for crit in [1, 2]:
#     for d in [4, 5, 6]:
#         for msl in [5, 10, 30]:
#             for mf in [3, 6, 12]:
#                 for n_est in [10]:
#                     rfc = RandomForestClassifier(criterion=crit, max_depth=d, min_samples_leaf=msl,
#                                                        max_features=mf, n_estimators=n_est)
#                     rfc.fit(X_train, y_sex_train)
#                     acc =  np.mean(rfc.predict(X_val) == y_sex_val)
#                     res[i,:] =[crit, d , msl, mf, n_est , acc]
#                     i+=1
#                     print(f'criterion = {crit}, ',f'depth = {d},', f'min_samples_leaf = {msl},', f'max_features = {mf}, ',
#                          f'n_estimators = {n_est}')
#                     print("Accuracy:", np.mean(rfc.predict(X_val) == y_sex_val))

# print(res[np.argmax(res[:,-1])])

In [19]:
# sort_res = res[np.argsort(res[:,-1])][::-1]
# sort_res[:10,:]

In [20]:
rfc = RandomForestClassifier(criterion='gini', max_depth=6, min_samples_leaf=10,
                             max_features="auto", n_estimators=50)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
importance = feature_importance(rfc)
k = 20
names, imp, ind = most_important_features(importance, features, k)
for i in range(k):
    print(str(i+1) + ".", names[i], imp[i])

Accuracy: 0.8234552332912989
Most important features:
1. 40kg 0.015175897291217887
2. mudakoff 0.014492404988749501
3. girlmeme 0.008919733291669264
4. zerofat 0.008896433570138324
5. 9o_6o_9o 0.008485462289480598
6. rapnewrap 0.008342413972604934
7. igm 0.00682615388348498
8. cook_good 0.005384617115125876
9. be.beauty 0.004251458593657704
10. bon 0.0040182957713958325
11. modnailru 0.00384995237427342
12. i_d_t 0.0035971063086401166
13. woman.blog 0.0035874210967665344
14. be.women 0.003568631166180023
15. beauty 0.003518277064544324
16. femalemem 0.003367311815130756
17. sh.cook 0.0032681873730482634
18. thesmolny 0.0031666768845773174
19. reflexia_our_feelings 0.002968639869080234
20. fuck_humor 0.00291928022332036


### CatBoost
В качестве альтернативы попробуем CatBoost. 

Установить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [21]:
from catboost import CatBoostClassifier, Pool
X, y = synthetic_dataset(1000)

model = CatBoostClassifier(iterations=5,
                           learning_rate=0.05,
                           depth=5,
                           loss_function='MultiClass')
model.fit(X, y, verbose=True)
print("Accuracy:", np.mean(model.predict(X) ==  y.reshape(1000,1)))
print("Importance:", model.get_feature_importance())

0:	learn: 1.0074859	total: 83.8ms	remaining: 335ms
1:	learn: 0.9254973	total: 86.9ms	remaining: 130ms
2:	learn: 0.8557739	total: 89.9ms	remaining: 60ms
3:	learn: 0.7937512	total: 92.9ms	remaining: 23.2ms
4:	learn: 0.7364863	total: 95.7ms	remaining: 0us
Accuracy: 1.0
Importance: [5.03193201e-04 8.11686844e-04 2.81763728e+01 2.81780796e+01
 4.36441847e+01 4.79515220e-05]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [22]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [23]:
model_age = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=4,
                           random_seed=42,
                           loss_function='MultiClass')
model_age.fit(X_train,y_age_train, verbose=False, eval_set=(X_eval, y_age_eval))

print("Accuracy:", np.mean(model_age.predict(X_test) == y_age_test.reshape(y_age_test.shape[0],1)))
importance = model_age.get_feature_importance()

print("Most important features:")
k = 20
names, imp,ind = most_important_features(importance, features, k)
for i in range(k):
    print(str(i+1) + ".", names[i])

Accuracy: 0.7351828499369483
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4. rhymes
5. mudakoff
6. leprum
7. bestad
8. xfilm
9. bot_maxim
10. rapnewrap
11. iwantyou
12. dayvinchik
13. pravdashowtop
14. tumblr_vacuum
15. fuck_humor
16. dzenpub
17. thesmolny
18. lixie
19. i_d_t
20. borsch


#### Пол

In [24]:
model_sex = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=4,
                           random_seed=42,
                           loss_function='MultiClass')
model_sex.fit(X_train,y_sex_train, verbose=False, eval_set=(X_eval, y_sex_eval))

print("Accuracy:", np.mean(model_sex.predict(X_test) == y_sex_test.reshape(y_sex_test.shape[0],1)))
importance = model_sex.get_feature_importance()

print("Most important features:")
k = 20
names, imp, ind = most_important_features(importance, features, k)
for i in range(k):
    print(str(i+1) + ".", names[i])

Accuracy: 0.8839848675914249
Most important features:
1. 40kg
2. modnailru
3. girlmeme
4. i_d_t
5. mudakoff
6. 9o_6o_9o
7. zerofat
8. femalemem
9. igm
10. academyofman
11. be.beauty
12. sh.cook
13. thesmolny
14. reflexia_our_feelings
15. be.women
16. woman.blog
17. h.made
18. recipes40kg
19. rapnewrap
20. team
